In [61]:
import data_extract as de
import time
import pandas as pd






In [62]:
d = de.ACDB().get_temp_forecasts()

In [63]:
d.dtypes

FCID                 object
StartTime    datetime64[ns]
Value               float64
Type                 object
dtype: object

In [64]:
d

,FCID,StartTime,Value,Type
0,Coast,2016-01-01 00:00:00,49.8,TempForecasts
1,Coast,2016-01-01 01:00:00,49.0,TempForecasts
2,Coast,2016-01-01 02:00:00,49.0,TempForecasts
3,Coast,2016-01-01 03:00:00,48.2,TempForecasts
4,Coast,2016-01-01 04:00:00,48.0,TempForecasts
...,...,...,...,...
2401507,West,2023-01-26 19:00:00,46.0,TempForecasts
2401508,West,2023-01-26 20:00:00,43.0,TempForecasts
2401509,West,2023-01-26 21:00:00,40.2,TempForecasts
2401510,West,2023-01-26 22:00:00,37.6,TempForecasts
